# ETL Process for Retail Data Warehouse

This notebook extracts raw retail sales data, transforms it into a star-schema format, 
and loads it into an SQLite database for OLAP analysis.

# 1. Imports

In [ ]:
import os, sqlite3, random
import numpy as np
import pandas as pd


# 2. Paths

In [ ]:
ETL_DIR = os.getcwd()
CSV_PATH = os.path.join(ETL_DIR, "synthetic_online_retail.csv")
DB_PATH  = os.path.join(ETL_DIR, "retail_dw.db")

CURRENT_DATE = pd.Timestamp("2025-08-13")
ONE_YEAR_AGO = CURRENT_DATE - pd.Timedelta(days=365)

print("Notebook folder:", ETL_DIR)
print("CSV will be at   :", CSV_PATH)
print("DB will be at    :", DB_PATH)

# 3. Generate CSV